# Description

Notebook for multiple covariate simulation results (Section 5).

# Imports

In [1]:
# to hide warnings for pretty notebook rendering in repo
import warnings
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import multiprocessing
import numpy as np
import os
import pandas as pd
import pingouin as pg


import pickle
import seaborn as sns
import sklearn
import sys
from tqdm import tqdm


# user imports
sys.path.append("../src/rdd-discovery/")

from utils.sim import *
from utils.rddd import *
from utils.pwr import *

# Load data

In [2]:
all_pwr_dict = pickle.load(open("../data/multdim_pwr_xgrid.dict", "rb"))

In [3]:
n_feats = [2, 4, 8, 16]
for n_feat in n_feats:
    pwr_dict = all_pwr_dict[n_feat]
    print("# feats: {}".format(n_feat))
    for label in ["all", "max"]:
        print("lower {} power: {:.3f} +/- {:.3f}".format(
            label,
            np.mean(pwr_dict["lower_{}".format(label)]),
            np.std(pwr_dict["lower_{}".format(label)])
        ))
        
        print("upper {} power: {:.3f} +/- {:.3f}".format(
            label,
            np.mean(pwr_dict["upper_{}".format(label)]),
            np.std(pwr_dict["upper_{}".format(label)])
        ))

# feats: 2
lower all power: 0.794 +/- 0.161
upper all power: 0.792 +/- 0.155
lower max power: 0.839 +/- 0.155
upper max power: 0.826 +/- 0.154
# feats: 4
lower all power: 0.777 +/- 0.149
upper all power: 0.776 +/- 0.157
lower max power: 0.813 +/- 0.144
upper max power: 0.809 +/- 0.154
# feats: 8
lower all power: 0.752 +/- 0.151
upper all power: 0.764 +/- 0.155
lower max power: 0.781 +/- 0.148
upper max power: 0.790 +/- 0.152
# feats: 16
lower all power: 0.747 +/- 0.149
upper all power: 0.750 +/- 0.155
lower max power: 0.783 +/- 0.148
upper max power: 0.766 +/- 0.156


# Oracle power

In [4]:
take = 0.05
alpha = 0.05 / len(np.arange(0.05, 0.96, 0.05)) 
lower_cutoff = 0.25
upper_cutoff = 0.75
n_samples = 1000
fuzzy_gap = 0.5
const = 0.2

## Baseline

In [5]:
u_ptake_lower = const + (take*upper_cutoff) + fuzzy_gap/2
u_ptake_upper = const + (take*upper_cutoff) + fuzzy_gap

l_ptake_lower = const + (take*lower_cutoff)
l_ptake_upper = const + (take*lower_cutoff) + fuzzy_gap/2


l_var = first_stage_asymp_var(l_ptake_lower, l_ptake_upper, n_samples, n_samples//2)
u_var = first_stage_asymp_var(u_ptake_lower, u_ptake_upper, n_samples, n_samples//2)


bl_lower_pwr = rdd_power(fuzzy_gap/2, var=l_var, alpha=alpha)
bl_upper_pwr = rdd_power(fuzzy_gap/2, var=u_var, alpha=alpha)

print("lower baseline power: {:.3f}".format(bl_lower_pwr))
print("upper baseline power: {:.3f}".format(bl_upper_pwr))

lower baseline power: 0.523
upper baseline power: 0.484


## With heterogeneity

In [6]:
u_ptake_lower = const + (take*upper_cutoff) + fuzzy_gap
u_ptake_upper = const + (take*upper_cutoff) + fuzzy_gap

l_ptake_lower = const + (take*lower_cutoff)
l_ptake_upper = const + (take*lower_cutoff) + fuzzy_gap


l_var = first_stage_asymp_var(l_ptake_lower, l_ptake_upper, n_samples, n_samples//4)
u_var = first_stage_asymp_var(u_ptake_lower, u_ptake_upper, n_samples, n_samples//4)

print("lower baseline power: {:.3f}".format(rdd_power(fuzzy_gap, var=l_var, alpha=alpha)))
print("upper baseline power: {:.3f}".format(rdd_power(fuzzy_gap, var=u_var, alpha=alpha)))

lower baseline power: 0.942
upper baseline power: 0.931


# t-tests against baseline oracle power

In [7]:
n_feats = [2, 4, 8, 16]
for n_feat in n_feats:
    pwr_dict = all_pwr_dict[n_feat]
    print("# feats: {}".format(n_feat))
    for label in ["lower", "upper"]:
        if label == "lower":
            display(pg.ttest(pwr_dict["{}_all".format(label)], bl_lower_pwr))
        else:
            display(pg.ttest(pwr_dict["{}_all".format(label)], bl_upper_pwr))

# feats: 2


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,43.564508,664,two-sided,7.493123e-197,"[0.78, 0.81]",1.689359,6.158e+192,1.0


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,50.458868,646,two-sided,2.717053e-226,"[0.78, 0.8]",1.983743,1.44e+222,1.0


# feats: 4


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,39.210551,531,two-sided,6.474441e-159,"[0.76, 0.79]",1.699993,8.845e+154,1.0


...

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,31.938334,443,two-sided,5.363356e-117,"[0.74, 0.77]",1.515726,1.474e+113,1.0


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,35.671418,394,two-sided,1.913488e-125,"[0.75, 0.78]",1.794824,3.778e+121,1.0


# feats: 16


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,29.994981,397,two-sided,4.390443e-104,"[0.73, 0.76]",1.503513,2.03e+100,1.0


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,30.916705,323,two-sided,1.566792e-98,"[0.73, 0.77]",1.717595,5.937e+94,1.0
